In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib
import pickle
from sklearn.model_selection import train_test_split

import warnings

warnings.filterwarnings("ignore")

from ti import *

# Load data (SPY only)

In [2]:
# # load the xlsx file
# def load_xlsx(file):
#     # read the xlsx file from the 5th line with header, only keep the first ns
#     df = pd.read_excel(file, header=5, usecols=[0, 1, 2, 3, 4, 5, 6])
#     return df

# data = load_xlsx("./data/stocks.xlsx")
# # only keep the 1st and 2nd column, name the first one as "Date" and the second one as "Price"
# data.columns = ["date", "close", "open", "low", "high", "3m_iv", "short_interest_rate"]


# # data.dropna(inplace=True)
# data.drop_duplicates(inplace=True)
# # data[['3m_iv', 'short_interest_rate']].ffill(inplace=True)

# # convert the "Date" column to datetime
# data["date"] = pd.to_datetime(data["date"])
# print(data.head())
# print(data.tail())

In [3]:
# data.to_csv(
#     './data/spy.csv'
# )

# Load from new csv

In [18]:
df = pd.read_csv("./data/spy.csv").iloc[:, 1:]

df

,date,close,open,low,high,3m_iv,short_interest_rate
0,2006-02-07,125.48,126.30,125.400,126.660,NaN,NaN
1,2006-02-08,126.62,125.88,125.600,126.780,NaN,NaN
2,2006-02-09,126.41,126.85,126.370,127.600,NaN,NaN
3,2006-02-10,126.64,126.42,125.450,127.130,NaN,NaN
4,2006-02-13,126.41,126.57,125.950,126.790,NaN,NaN
...,...,...,...,...,...,...,...
4561,2024-03-22,521.21,522.11,520.970,522.605,12.6662,NaN
4562,2024-03-25,519.77,519.80,519.610,520.950,12.6790,NaN
4563,2024-03-26,518.81,521.23,518.400,521.580,12.7001,NaN
4564,2024-03-27,523.17,521.71,519.485,523.210,12.6752,NaN


In [19]:
df.columns

Index(['date', 'close', 'open', 'low', 'high', '3m_iv', 'short_interest_rate'], dtype='object')

# Selected df

In [20]:
sele_df = df[["date", "close", "open", "low", "high"]].copy()

sele_df.dropna(inplace=True)

sele_df["date"] = pd.to_datetime(sele_df["date"])

# sele_df = sele_df[(sele_df['date'] >= '2015-01-01') & (sele_df['date'] <= '2019-12-31')]

sele_df.set_index("date", inplace=True)

sele_df

,close,open,low,high
date,,,,
2006-02-07,125.48,126.30,125.400,126.660
2006-02-08,126.62,125.88,125.600,126.780
2006-02-09,126.41,126.85,126.370,127.600
2006-02-10,126.64,126.42,125.450,127.130
2006-02-13,126.41,126.57,125.950,126.790
...,...,...,...,...
2024-03-22,521.21,522.11,520.970,522.605
2024-03-25,519.77,519.80,519.610,520.950
2024-03-26,518.81,521.23,518.400,521.580


# Lags and Rolling Window Length

In [21]:
lags_period = [1, 3, 5, 8, 21]

# Set 1: Only price data 
Open, high, low, close

In [30]:
# Set X and y
X = sele_df[["close", "open", "low", "high"]][
    (sele_df.index >= "2015-01-01") & (sele_df.index <= "2019-12-31")
]
y = sele_df["close"][(sele_df.index >= "2015-01-01") & (sele_df.index <= "2019-12-31")]

In [31]:
def add_lagged_features(X, lags_period):
    lag_X = pd.DataFrame()
    for lag in lags_period:
        for column in np.unique(X.columns):
            lag_X[f"{column}_lag_{lag}"] = X[column].shift(lag)
    return lag_X.dropna()


def prepare_X_y(X, y, lags_period, test_size=0.2):
    if lags_period != None:
        # Add lagged features
        lag_X = add_lagged_features(X, lags_period)
        # Align target variable y with the rows in lag_X
        y_aligned = y.loc[lag_X.index]
    else:
        lag_X = X
        y_aligned = y

    # Calculate split index
    split_idx = int(len(lag_X) * (1 - test_size))

    # Split into training and testing sets based on the calculated index
    X_train = lag_X.iloc[:split_idx]
    X_test = lag_X.iloc[split_idx:]
    y_train = y_aligned.iloc[:split_idx]
    y_test = y_aligned.iloc[split_idx:]

    return (X_train, X_test, y_train, y_test)

In [32]:
only_price = prepare_X_y(X, y, lags_period, test_size=0.2)

print("train time period")
print(only_price[0].index[0], only_price[0].index[-1])
print("test time period")
print(only_price[1].index[0], only_price[1].index[-1])


# pickle_file_path = "./data/only_price.pkl"  # Adjust path as necessary
# with open(pickle_file_path, "wb") as file:
#     pickle.dump(only_price, file)

# print(f"Datasets saved to {pickle_file_path}")

train time period
2015-02-03 00:00:00 2019-01-07 00:00:00
test time period
2019-01-08 00:00:00 2019-12-31 00:00:00


In [33]:
only_price[0]

,close_lag_1,high_lag_1,low_lag_1,open_lag_1,close_lag_3,high_lag_3,low_lag_3,open_lag_3,close_lag_5,high_lag_5,low_lag_5,open_lag_5,close_lag_8,high_lag_8,low_lag_8,open_lag_8,close_lag_21,high_lag_21,low_lag_21,open_lag_21
date,,,,,,,,,,,,,,,,,,,,
2015-02-03,201.92,202.03,197.86,200.05,201.9902,202.30,198.68,200.38,202.7400,204.1200,201.74,202.97,206.10,206.2600,202.33,203.99,205.43,206.88,204.1800,206.38
2015-02-04,204.84,204.85,202.55,203.00,199.4500,202.17,199.13,200.57,200.1400,204.2900,199.91,204.17,204.97,206.1000,204.81,205.79,201.72,204.37,201.3500,204.17
2015-02-05,204.06,205.38,203.51,203.92,201.9200,202.03,197.86,200.05,201.9902,202.3000,198.68,200.38,205.45,205.5600,203.85,204.71,199.82,202.72,198.8550,202.09
2015-02-06,206.12,206.30,204.77,204.86,204.8400,204.85,202.55,203.00,199.4500,202.1700,199.13,200.57,202.74,204.1200,201.74,202.97,202.31,202.72,200.8800,201.42
2015-02-09,205.55,207.24,204.92,206.56,204.0600,205.38,203.51,203.92,201.9200,202.0300,197.86,200.05,200.14,204.2900,199.91,204.17,205.90,206.16,203.9900,204.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31,247.75,251.40,246.45,249.58,246.1800,246.18,233.76,235.97,240.7000,249.7100,239.98,246.74,255.08,257.9500,253.28,257.20,274.58,274.58,268.3272,269.60
2019-01-02,249.92,250.19,247.47,249.56,248.0700,248.29,238.96,242.57,234.3400,240.8355,234.27,239.04,251.26,259.4000,249.35,255.17,273.98,275.55,272.4300,273.71
2019-01-03,250.18,251.21,245.95,245.98,247.7500,251.40,246.45,249.58,246.1800,246.1800,233.76,235.97,247.17,251.6200,244.65,249.86,275.65,276.28,273.4500,273.81


# Set 2: Price and Technical Indicators

In [34]:
from ti import *

In [35]:
sele_ti_df = (
    pd.concat([sele_df, pd.concat(technical_indicators(sele_df), axis=1)], axis=1)
    .T.drop_duplicates()
    .T
)
sele_ti_df

,close,open,low,high,ADX,APO,AROONOSC,BOP,CCI,CMO,...,BBANDS_LOWER,ATR,NATR,TRANGE,CHANDELIER_EXIT_LONG,CHANDELIER_EXIT_SHORT,HT_DCPERIOD,HT_PHASOR_inphase,HT_PHASOR_quadrature,HT_TRENDMODE
date,,,,,,,,,,,,,,,,,,,,,
2006-02-07,125.48,126.30,125.400,126.660,NaN,NaN,NaN,-0.650794,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2006-02-08,126.62,125.88,125.600,126.780,NaN,NaN,NaN,0.627119,NaN,NaN,...,NaN,NaN,NaN,1.300,NaN,NaN,NaN,NaN,NaN,0.0
2006-02-09,126.41,126.85,126.370,127.600,NaN,NaN,NaN,-0.357724,NaN,NaN,...,NaN,NaN,NaN,1.230,NaN,NaN,NaN,NaN,NaN,0.0
2006-02-10,126.64,126.42,125.450,127.130,NaN,NaN,NaN,0.130952,NaN,NaN,...,NaN,NaN,NaN,1.680,NaN,NaN,NaN,NaN,NaN,0.0
2006-02-13,126.41,126.57,125.950,126.790,NaN,NaN,NaN,-0.190476,NaN,NaN,...,NaN,NaN,NaN,0.840,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-22,521.21,522.11,520.970,522.605,25.962336,3.031429,85.714286,-0.550459,151.403298,31.960840,...,503.261697,4.623625,0.887094,1.635,510.370636,516.759364,24.069800,-1.721653,-5.978936,1.0
2024-03-25,519.77,519.80,519.610,520.950,25.802733,2.877143,85.714286,-0.022388,99.382761,26.909109,...,504.043488,4.407651,0.848000,1.600,510.776971,518.083029,25.258255,-2.925568,5.839131,1.0
2024-03-26,518.81,521.23,518.400,521.580,25.361118,2.857500,28.571429,-0.761006,73.773353,23.514373,...,504.883973,4.319962,0.832667,3.180,510.949381,517.910619,25.451842,1.475988,15.584334,1.0


In [38]:
X_ti = sele_ti_df[
    (sele_ti_df.index >= "2015-01-01") & (sele_ti_df.index <= "2019-12-31")
]
y_ti = sele_ti_df["close"][
    (sele_ti_df.index >= "2015-01-01") & (sele_ti_df.index <= "2019-12-31")
]

In [39]:
price_ti = prepare_X_y(X_ti, y_ti, lags_period, test_size=0.2)

print("train time period")
print(price_ti[0].index[0], price_ti[0].index[-1])
print("test time period")
print(price_ti[1].index[0], price_ti[1].index[-1])


pickle_file_path = "./data/price_ti.pkl"  # Adjust path as necessary
with open(pickle_file_path, "wb") as file:
    pickle.dump(price_ti, file)

print(f"Datasets saved to {pickle_file_path}")

train time period
2015-02-03 00:00:00 2019-01-07 00:00:00
test time period
2019-01-08 00:00:00 2019-12-31 00:00:00
Datasets saved to ./data/price_ti.pkl


# ***New (stocks_long.csv)***

In [34]:
df = pd.read_csv("./data/stocks_long.csv").ffill()

df["DATE"] = pd.to_datetime(df["DATE"])

df.set_index("DATE", inplace=True)

# df.dropna(inplace=True)

df.rename(
    columns={
        "SPY_HIGH": "high",
        "SPY_LAST": "close",
        "SPY_LOW": "low",
        "SPY_OPEN": "open",
    },
    inplace=True,
)

df

,SPY_3M IMPLIED VOL,high,close,low,open,SPY_SHORT INTEREST RATIO,XBI_3M IMPLIED VOL,XBI_HIGH,XBI_LAST,XBI_LOW,...,XOP_OPEN,XOP_SHORT INTEREST RATIO,XRT_3M IMPLIED VOL,XRT_HIGH,XRT_LAST,XRT_LOW,XRT_OPEN,XRT_SHORT INTEREST RATIO,tweet_sentiment,news_sentiment
DATE,,,,,,,,,,,,,,,,,,,,,
2006-02-06,NaN,126.800,126.60,126.170,126.44,NaN,NaN,16.5000,16.4167,16.4000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-02-07,NaN,126.660,125.48,125.400,126.30,NaN,NaN,16.5200,16.3667,16.3667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-02-08,NaN,126.780,126.62,125.600,125.88,NaN,NaN,16.3600,16.3600,16.2667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-02-09,NaN,127.600,126.41,126.370,126.85,NaN,NaN,16.7233,16.5133,16.5133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-02-10,NaN,127.130,126.64,125.450,126.42,NaN,NaN,16.5600,16.5600,16.3300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-22,12.6662,522.605,521.21,520.970,522.11,1.418,30.7593,95.3800,93.5600,93.5000,...,151.41,4.046,24.8288,78.08,77.23,77.20,77.83,3.536,0.036756,0.164516
2024-03-25,12.6790,520.950,519.77,519.610,519.80,1.418,30.7668,94.4700,92.6700,92.4950,...,151.43,4.046,24.5762,77.94,76.54,76.53,77.44,3.536,0.036756,0.164516
2024-03-26,12.7001,521.580,518.81,518.400,521.23,1.418,30.7501,94.4600,92.6900,92.6600,...,152.60,4.046,24.4271,77.33,76.84,76.65,76.82,3.536,0.036756,0.164516


In [35]:
sele_ti_df = (
    pd.concat([df, pd.concat(technical_indicators(df), axis=1)], axis=1)
    .T.drop_duplicates()
    .T
)
sele_ti_df

,SPY_3M IMPLIED VOL,high,close,low,open,SPY_SHORT INTEREST RATIO,XBI_3M IMPLIED VOL,XBI_HIGH,XBI_LAST,XBI_LOW,...,BBANDS_LOWER,ATR,NATR,TRANGE,CHANDELIER_EXIT_LONG,CHANDELIER_EXIT_SHORT,HT_DCPERIOD,HT_PHASOR_inphase,HT_PHASOR_quadrature,HT_TRENDMODE
DATE,,,,,,,,,,,,,,,,,,,,,
2006-02-06,NaN,126.800,126.60,126.170,126.44,NaN,NaN,16.5000,16.4167,16.4000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2006-02-07,NaN,126.660,125.48,125.400,126.30,NaN,NaN,16.5200,16.3667,16.3667,...,NaN,NaN,NaN,1.260,NaN,NaN,NaN,NaN,NaN,0.0
2006-02-08,NaN,126.780,126.62,125.600,125.88,NaN,NaN,16.3600,16.3600,16.2667,...,NaN,NaN,NaN,1.300,NaN,NaN,NaN,NaN,NaN,0.0
2006-02-09,NaN,127.600,126.41,126.370,126.85,NaN,NaN,16.7233,16.5133,16.5133,...,NaN,NaN,NaN,1.230,NaN,NaN,NaN,NaN,NaN,0.0
2006-02-10,NaN,127.130,126.64,125.450,126.42,NaN,NaN,16.5600,16.5600,16.3300,...,NaN,NaN,NaN,1.680,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-22,12.6662,522.605,521.21,520.970,522.11,1.418,30.7593,95.3800,93.5600,93.5000,...,503.261697,4.623625,0.887094,1.635,510.370636,516.759364,24.069800,-1.721653,-5.978936,1.0
2024-03-25,12.6790,520.950,519.77,519.610,519.80,1.418,30.7668,94.4700,92.6700,92.4950,...,504.043488,4.407651,0.848000,1.600,510.776971,518.083029,25.258255,-2.925568,5.839131,1.0
2024-03-26,12.7001,521.580,518.81,518.400,521.23,1.418,30.7501,94.4600,92.6900,92.6600,...,504.883973,4.319962,0.832667,3.180,510.949381,517.910619,25.451842,1.475988,15.584334,1.0


In [36]:
sele_ti_df.to_csv("stocks_long_w_ti")